In [39]:
import os
os.chdir('../')
import dataset.aGPTset.ExpressiveGuitarTechniquesDataset as agptset


dataset = agptset.import_db()


  instruments_df.fillna(np.nan).replace([np.nan], [None], inplace=True) # Replace NaN with None


In [40]:
# First of all we need to add a "technique_id" to tot_files_rows and tot_files_rows since id is missing
tot_files_rows = dataset['files_df'].copy()

names_for_techniques = {
    "0": ['_kick_','class0'],
    "1": ['_lowersidefingers_','class1'],
    "2": ['_thumb_','class2'],
    "3": ['_keybed_','class3'],
    "4": ['_naturalharmonics_','class4'],
    "5": ['_palmmute_','class5'],
    "6": ['_picknearbridge_','class6'],
    "7": ['_soundholepick_','class7'],
    "8": ['bending_'],
    "9": ['_hammeron_'],
    "10":['_staccato_'],
    "11":['_vibrato_']
}

# add empty column
tot_files_rows['techid'] = None
for tech_id, tech_names in names_for_techniques.items():
    for tech_name in tech_names:
        tot_files_rows['techid'] = tot_files_rows['techid'].mask(tot_files_rows.index.str.contains(tech_name), tech_id)

# print(tot_files_rows['techid'])
# print all filenames that still have None techid
for e in tot_files_rows[tot_files_rows['techid'].isnull()].index:
    assert 'impro' in e, f"Error: {e} has no techid and is not an improvisation with many techniques"

In [50]:
labeled_files = dataset['noteLabels_df'].index.levels[1]
# From dataset['files_df'] select all rows with index in labeled_files
labeled_files_rows = tot_files_rows.loc[labeled_files].copy()

# labeled_files_rows

In [51]:
perfileCounts = dataset['noteLabels_df'].groupby(dataset['noteLabels_df'].index.get_level_values(1)).size()
for lr in labeled_files_rows.index:
    labeled_files_rows.at[lr, 'Labeled Notes'] = perfileCounts[lr].astype(int)

## 1. Instruments Table

In [52]:
# seconds_to_hms = lambda x: f"{int(x//3600):02d}h"+" "+"{int((x%3600)//60):02d}'"+" "+"{int(x%60):02d}''"
def seconds_to_hms(x, drop_seconds=True):
    res = ''
    if x // 3600:
        res += f"{int(x//3600):2d}h"
    else:
        res += '   '
    res += ' '
    if x // 60:
        res += f"{int((x%3600)//60):02d}'"
    else:
        res += '   '
    if not drop_seconds:
        res += ' '
        res += f"{int(x%60):2d}''"

    return res
    # return f"{int(x//3600):02d}h"+" "+"{int((x%3600)//60):02d}'"+" "+"{int(x%60):02d}''"

# seconds_to_hms(3639)

In [109]:
instrumentsdf = dataset['instruments_df'].copy()

# Drop column in the next list
toremove = ['owner_id','instrument_type','additional_notes']

for col in toremove:
    if col in instrumentsdf.columns:
        instrumentsdf.drop(col, axis=1, inplace=True)

torename = {'has_condenser_microphone': 'C?*'}
for col in torename:
    if col in instrumentsdf.columns:
        instrumentsdf.rename(columns={col: torename[col]}, inplace=True)

# For all cells in column "pickup" replace factory_option with "Factory Option"
instrumentsdf['pickup'] = instrumentsdf['pickup'].replace('factory_option', 'Factory Option')

# Add column "Tot dataset time" with the sum of all the time in the dataset (from tot_files_rows)
# So for all rows take the guitar_id in index, take all the entries in tot_files_rows with that guitar_id and sum the "duration" column
DO_ADD_TOT_DURATION = True
if DO_ADD_TOT_DURATION:
    tot_files_rows['duration_f'] = tot_files_rows['duration'].apply(lambda x: float(x.split()[0]))
    instrumentsdf['Tot Rec. Time'] = tot_files_rows.groupby('guitar_id')['duration_f'].sum()

    sumOfAll = tot_files_rows['duration_f'].sum()
    # print(f"Sum of all durations: {sumOfAll}, which in HH:MM:SS is {sumOfAll//3600}h{(sumOfAll%3600)//60}'{sumOfAll%60}''")

    # instrumentsdf['Tot Rec. Time'] is in float seconds, convert it to HHh MM'SS''
    # add percentage of total time
    instrumentsdf['Perc Of Tot'] = instrumentsdf['Tot Rec. Time']/sumOfAll*100
    instrumentsdf['Tot Rec. Time'] = instrumentsdf['Tot Rec. Time'].apply(seconds_to_hms)
    # Now append the percentage and drop the float column
    instrumentsdf['Tot Rec. Time'] = instrumentsdf['Tot Rec. Time'] + ' (' + instrumentsdf['Perc Of Tot'].apply(lambda x: f"{x:.1f}%") + ')'
    instrumentsdf.drop('Perc Of Tot', axis=1, inplace=True)

DO_ADD_LABELED_DURATION=True
if DO_ADD_LABELED_DURATION:
    labeled_files_rows['duration_f'] = labeled_files_rows['duration'].apply(lambda x: float(x.split()[0]))
    instrumentsdf['Labeled Rec. Time'] = labeled_files_rows.groupby('guitar_id')['duration_f'].sum()
    sumOfAll = labeled_files_rows['duration_f'].sum()
    instrumentsdf['Perc Of Labeled'] = instrumentsdf['Labeled Rec. Time']/sumOfAll*100
    instrumentsdf['Labeled Rec. Time'] = instrumentsdf['Labeled Rec. Time'].apply(seconds_to_hms)
    instrumentsdf['Labeled Rec. Time'] = instrumentsdf['Labeled Rec. Time'] + ' (' + instrumentsdf['Perc Of Labeled'].apply(lambda x: f"{x:.1f}%") + ')'
    instrumentsdf.drop('Perc Of Labeled', axis=1, inplace=True)
    # labeled_files_rows['duration_f'].sum()


# In column "Has Condenser Microphone" replace "True" with "\cmark" and "False" with "\xmark"
instrumentsdf['C?*'] = instrumentsdf['C?*'].replace({'true': r'\cmark', 'false': r'\xmark'})

# # Sort by "brand" column
# instrumentsdf.sort_values(by='brand', inplace=True)
# # Reset index and start from 1
instrumentsdf.index.name='ID'	
instrumentsdf.reset_index(drop=False, inplace=True)
# instrumentsdf.index += 1

# Replace "LR baggs anthem acoustic microphone system" with just "LR Baggs Anthem"
instrumentsdf['pickup'] = instrumentsdf['pickup'].replace('LR baggs anthem acoustic microphone system', 'LR Baggs Anthem')

#Escape % for latex
instrumentsdf = instrumentsdf.map(lambda x: x.replace('%', '\%'))

# All header  first letter capital
instrumentsdf.columns = instrumentsdf.columns.str.capitalize()
# All header bold
instrumentsdf.columns = [f"\\textbf{{{col}}}" for col in instrumentsdf.columns]




latextb1 = instrumentsdf.to_latex(index=False, escape=False, column_format='llllcrr')

latextb1 = r'\begin{adjustbox}{width=\textwidth}' + '\n' + latextb1  + r'\end{adjustbox}'

bottomnote = r'''\raggedright
\small
\textit{*C?: Has condenser microphone. If true, the condenser microphone was set to a 50\% blend with the piezoelectric transducer for recording. If not, only the piezoelectric pickup was used.}
'''

# endTabPos = latextb1.find(r'\end{tabular}')
# latextb1 = latextb1[:endTabPos] + bottomnote + latextb1[endTabPos:]
latextb1 = latextb1 + '\n'+ bottomnote


# Append \hspace{.52em} before all  \(\d\.\d\%\) 
import re
# latextb1 = re.sub(r'\(\d\.\d\\%\)', r'$1', latextb1)
# Redo but actually append CIAO to text without modifying it
finds = re.findall(r'\(\d\.\d\\%\)', latextb1)
for f in finds:
    latextb1 = latextb1.replace(f, r'\hspace{.52em}'+f)

print(latextb1)

\begin{adjustbox}{width=\textwidth}
\begin{tabular}{llllcrr}
\toprule
\textbf{Id} & \textbf{Brand} & \textbf{Model} & \textbf{Pickup} & \textbf{C?*} & \textbf{Tot rec. time} & \textbf{Labeled rec. time} \\
\midrule
0 & Eko & unknown & Factory Option & \cmark &  2h 54' (18.3\%) &  1h 42' (16.9\%) \\
1 & Eko & WOW 018 KOA & Fishman Flex Blend & \cmark &  2h 44' (17.2\%) &  1h 20' (13.3\%) \\
2 & Taylor & 114 CE & LR Baggs Anthem & \cmark &  2h 53' (18.1\%) &  1h 27' (14.4\%) \\
3 & Maton & EBG-808 & AP mic & \cmark &  2h 30' (15.8\%) &  1h 21' (13.5\%) \\
4 & Yamaha & APX-8A & Factory Option & \xmark &  1h 50' (11.6\%) &  1h 18' (13.1\%) \\
5 & Crafter & TB-G-1000 & Factory Option & \cmark &  1h 31' \hspace{.52em}(9.6\%) &  1h 23' (13.9\%) \\
6 & Crafter & GLXE-4000/RS & Factory Option & \cmark &  1h 30' \hspace{.52em}(9.5\%) &  1h 30' (15.0\%) \\
\bottomrule
\end{tabular}
\end{adjustbox}
\raggedright
\small
\textit{*C?: Has condenser microphone. If true, the condenser microphone was set

## 2. Techniques explaination table

In [90]:
expressiveTechniques_df = dataset['expressiveTechniques_df'].copy()
# Drop name_nospace column 
expressiveTechniques_df.drop('name_nospace', axis=1, inplace=True)
#  Remove name of index
expressiveTechniques_df.index.name = None


# Compute duration (hh:mm:ss) for each technique, both in the tot dataset and in the labeled portion



# labeled_files_rows = tot_files_rows.loc[labeled_files]

DO_ADD_TECH_DURATION_TOT = True
if DO_ADD_TECH_DURATION_TOT:
    tot_files_rows['duration_f'] = tot_files_rows['duration'].apply(lambda x: float(x.split()[0]))
    # display(tot_files_rows.groupby('techid')['duration_f'].sum())
    #Put into expressiveTechniques_df['Tot Rec. Time'] of the index that matches techid, the time of the sum of the duration_f column for that techid
    for tech_id in expressiveTechniques_df.index:
        # tech_id = int(tech_id)
        expressiveTechniques_df.at[tech_id, 'Tot Rec. Time'] = tot_files_rows[tot_files_rows['techid'] == tech_id]['duration_f'].sum()

    # print('Heeeyoo')
    # print(expressiveTechniques_df.index)
    # display(expressiveTechniques_df['Tot Rec. Time'])

    sumOfAll = expressiveTechniques_df['Tot Rec. Time'].sum()
    expressiveTechniques_df['Perc Of Tot'] = expressiveTechniques_df['Tot Rec. Time']/sumOfAll*100
    expressiveTechniques_df['Tot Rec. Time'] = expressiveTechniques_df['Tot Rec. Time'].apply(seconds_to_hms)
    expressiveTechniques_df['Tot Rec. Time'] = expressiveTechniques_df['Tot Rec. Time'] + ' (' + expressiveTechniques_df['Perc Of Tot'].apply(lambda x: f"{x:4.1f}%") + ')'
    expressiveTechniques_df.drop('Perc Of Tot', axis=1, inplace=True)

DO_ADD_TECH_LABELED_DURATION = True
if DO_ADD_TECH_LABELED_DURATION:
    labeled_files_rows['duration_f'] = labeled_files_rows['duration'].apply(lambda x: float(x.split()[0]))
    for tech_id in expressiveTechniques_df.index:
        expressiveTechniques_df.at[tech_id, 'Labeled Rec. Time'] = labeled_files_rows[labeled_files_rows['techid'] == tech_id]['duration_f'].sum()
    
    sumOfAll = labeled_files_rows['duration_f'].sum()
    expressiveTechniques_df['Perc Of Labeled'] = expressiveTechniques_df['Labeled Rec. Time']/sumOfAll*100
    expressiveTechniques_df['Labeled Rec. Time'] = expressiveTechniques_df['Labeled Rec. Time'].apply(seconds_to_hms)
    expressiveTechniques_df['Labeled Rec. Time'] = expressiveTechniques_df['Labeled Rec. Time'] + ' (' + expressiveTechniques_df['Perc Of Labeled'].apply(lambda x: f"{x:4.1f}%") + ')'
    
    # remove occurrences of '        00'' (0.0%)' and replace with '\xmark'

    XMARK = r'\textcolor{blue}{\footnotesize(Onsets not labeled)}'
    expressiveTechniques_df['Labeled Rec. Time'] = expressiveTechniques_df['Labeled Rec. Time'].str.replace('        00\'\' ( 0.0%)', XMARK)
    for eda in expressiveTechniques_df['Labeled Rec. Time']:
        if '( 0.0%)' in eda:
            expressiveTechniques_df['Labeled Rec. Time'] = expressiveTechniques_df['Labeled Rec. Time'].str.replace(eda, XMARK)
    # print(expressiveTechniques_df['Labeled Rec. Time'])

    # print(list(expressiveTechniques_df['Labeled Rec. Time']))
    # expressiveTechniques_df['Labeled Rec. Time'].str.replace("        00'' (0.0%)", '\\xmark')
    
    expressiveTechniques_df.drop('Perc Of Labeled', axis=1, inplace=True)


# Add number of notes (labeled_files_rows['Labeled Notes]) to expressiveTechniques_df
for tech_id in expressiveTechniques_df.index:
    # print(tech_id)
    # display(labeled_files_rows['Labeled Notes'])
    # labeled_files_rows[labeled_files_rows['techid'] == tech_id]['Labeled Notes'].sum()
    # expressiveTechniques_df.at[tech_id, 'Labeled Notes'] = "1" #labeled_files_rows[labeled_files_rows['techid'] == tech_id]['Labeled Notes'].sum()
    expressiveTechniques_df.at[tech_id, '\# Labeled Notes'] = labeled_files_rows[labeled_files_rows['techid'] == tech_id]['Labeled Notes'].sum()
    # forat to int string with thousands comma
    expressiveTechniques_df.at[tech_id, '\# Labeled Notes'] = f"{int(expressiveTechniques_df.at[tech_id, '\# Labeled Notes']):,}"
    
    # replace "0" with ""
    expressiveTechniques_df['\# Labeled Notes'] = expressiveTechniques_df['\# Labeled Notes'].replace('0', '')



# drop description column
expressiveTechniques_df.drop('description', axis=1, inplace=True)

# Turn index into int and sum 1
expressiveTechniques_df.index = expressiveTechniques_df.index.astype(int) + 1
expressiveTechniques_df.reset_index(inplace=True)
expressiveTechniques_df.rename(columns={'index': 'ID'}, inplace=True)
# turn into string
expressiveTechniques_df['ID'] = expressiveTechniques_df['ID'].astype(str)

# rename type as Type
expressiveTechniques_df.rename(columns={'type': 'Type'}, inplace=True)

# Bold header
expressiveTechniques_df.columns = [f"\\textbf{{{col}}}" for col in expressiveTechniques_df.columns]

#Escape % for latex
expressiveTechniques_df = expressiveTechniques_df.map(lambda x: x.replace('%', '\%'))

tech_lat = expressiveTechniques_df.to_latex(index=False, column_format='lllrrr')
# Replace "\begin{tabular}" with \begin{tabularx}{\textwidth} and "\end{tabular}" with \end{tabularx}
# tech_lat = tech_lat.replace('\\begin{tabular}', '\\begin{tabularx}{\\textwidth}')
# tech_lat = tech_lat.replace('\\end{tabular}', '\\end{tabularx}')

# Encase in \begin{adjustbox}{max width=\textwidth} and \end{adjustbox}
tech_lat = '\\begin{adjustbox}{width=\\textwidth}\n' + tech_lat + '\\end{adjustbox}'

print(tech_lat)

'\multicolumn{2}{c}{\textbf{Tot Rec. Time}} & \multicolumn{2}{c}{\textbf{Labeled Rec. Time}}'

\begin{adjustbox}{width=\textwidth}
\begin{tabular}{lllrrr}
\toprule
\textbf{ID} & \textbf{name} & \textbf{Type} & \textbf{Tot Rec. Time} & \textbf{Labeled Rec. Time} & \textbf{# Labeled Notes} \\
\midrule
1 & Kick & percussive &     20' ( 2.2\%) &     20' ( 3.3\%) & 1,950 \\
2 & Snare-A & percussive &     19' ( 2.0\%) &     19' ( 3.2\%) & 1,968 \\
3 & Tom & percussive &     19' ( 2.1\%) &     19' ( 3.3\%) & 2,167 \\
4 & Snare-B & percussive &     19' ( 2.1\%) &     19' ( 3.3\%) & 1,943 \\
5 & Natural Harmonics & pitched &     52' ( 5.6\%) &     52' ( 8.7\%) & 2,092 \\
6 & Palm Mute & pitched &  1h 49' (11.8\%) &  1h 49' (18.2\%) & 7,588 \\
7 & Pick Near Bridge & pitched &  2h 40' (17.1\%) &  2h 40' (26.5\%) & 7,227 \\
8 & Pick Over the Soundhole & pitched &  3h 22' (21.7\%) &  3h 22' (33.6\%) & 7,657 \\
9 & Bending & pitched &  1h 17' ( 8.3\%) & \textcolor{blue}{\footnotesize(Onsets not labeled)} &  \\
10 & Hammer-on & pitched &  1h 02' ( 6.7\%) & \textcolor{blue}{\footnotesize(Onsets

/tmp/ipykernel_2230327/2426643690.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1,950' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  expressiveTechniques_df.at[tech_id, '# Labeled Notes'] = f"{int(expressiveTechniques_df.at[tech_id, '# Labeled Notes']):,}"


## Dynamics

In [55]:
# group by intensity and sum time

# labeled_files_rows
# tot_files_rows

sumtime = labeled_files_rows.groupby('playing_dynamics_or_intensity')['duration_f'].sum()
sumall = labeled_files_rows['duration_f'].sum()
perc = labeled_files_rows.groupby('playing_dynamics_or_intensity')['duration_f'].sum()/sumall*100
perc = perc.apply(lambda x: f"{x:.1f}%")
perc = perc.rename('Perc Of Labeled')

# perc and sumtume into one df
perc_df = perc.to_frame()
perc_df['Labeled Rec. Time'] = sumtime.apply(seconds_to_hms)
perc_df['Labeled Rec. Time'] = perc_df['Labeled Rec. Time'] + ' (' + perc_df['Perc Of Labeled'] + ')'
perc_df.drop('Perc Of Labeled', axis=1, inplace=True)

# tot_files_rows
sumtime = tot_files_rows.groupby('playing_dynamics_or_intensity')['duration_f'].sum()
sumall = tot_files_rows['duration_f'].sum()
perc_df['Perc Of Tot'] = tot_files_rows.groupby('playing_dynamics_or_intensity')['duration_f'].sum()/sumall*100
perc_df['Tot Rec. Time'] = sumtime.apply(seconds_to_hms)
perc_df['Tot Rec. Time'] = perc_df['Tot Rec. Time'] + ' (' + perc_df['Perc Of Tot'].apply(lambda x: f"{x:.1f}%") + ')'
perc_df.drop('Perc Of Tot', axis=1, inplace=True)

perc_df.index.name=None

# reorder columns
perc_df = perc_df[['Tot Rec. Time', 'Labeled Rec. Time']]
#Escape % for latex
perc_df = perc_df.map(lambda x: x.replace('%', '\%'))
# bold column names
perc_df.columns = [f"\\textbf{{{col}}}" for col in perc_df.columns]
#Italic index
perc_df.index = [f"\\textit{{{idx}}}" for idx in perc_df.index]




perc_lat = perc_df.to_latex(index=True)

print(perc_lat)

\begin{tabular}{lll}
\toprule
 & \textbf{Tot Rec. Time} & \textbf{Labeled Rec. Time} \\
\midrule
\textit{forte} &  4h 55' (31.0\%) &  3h 05' (30.8\%) \\
\textit{mezzoforte} &  5h 53' (37.0\%) &  3h 48' (37.8\%) \\
\textit{piano} &  5h 00' (31.5\%) &  3h 09' (31.4\%) \\
\bottomrule
\end{tabular}



## Labelers table

In [56]:
# take labeled_files_rows and group by labeler_id, summing duration_f
sumtime = labeled_files_rows.groupby('labeler_id')['duration_f'].sum()
sumall = labeled_files_rows['duration_f'].sum()
perc = labeled_files_rows.groupby('labeler_id')['duration_f'].sum()/sumall*100
perc = perc.apply(lambda x: f"{x:.1f}%")
perc = perc.rename('Perc Of Labeled')

perc_df = perc.to_frame()
# perc_df.reset_index(inplace=True, drop=True)

# Repeat for # notes
sumnotes = labeled_files_rows.groupby('labeler_id')['Labeled Notes'].sum()
perc_df['Labeled Notes'] = sumnotes
perc_df['Perc Of Labeled Notes'] = perc_df['Labeled Notes']/sumnotes.sum()*100
perc_df['Perc Of Labeled Notes'] = perc_df['Perc Of Labeled Notes'].apply(lambda x: f"{x:.1f}\%")

# Convert ''Labeled Notes'' to int
perc_df['Labeled Notes'] = perc_df['Labeled Notes'].astype(int)

# To Latex
perc_df['Tot Rec. Time'] = sumtime.apply(seconds_to_hms)
# perc_df['Tot Rec. Time'] = perc_df['Tot Rec. Time'] + ' (' + perc_df['Perc Of Labeled'] + ')'
perc_df.drop('Perc Of Labeled', axis=1, inplace=True)

perc_df.index.name=None
perc_df = perc_df[['Tot Rec. Time', 'Labeled Notes', 'Perc Of Labeled Notes']]
# reorder with 'Tot Rec. Time' as last
perc_df = perc_df[['Labeled Notes', 'Perc Of Labeled Notes', 'Tot Rec. Time']]
# perc_df['Labeled Notes'] = perc_df['Labeled Notes'].apply() + ' (' + perc_df['Perc Of Labeled Notes'] + ')'
# perc_df.drop('Perc Of Labeled Notes', axis=1, inplace=True)
perc_df['Labeled Notes'] = perc_df['Labeled Notes'].apply(lambda x: f"{x:,}")
# merge labeled notes and (percentage) in one column
perc_df['Labeled Notes'] = perc_df['Labeled Notes'] + ' (' + perc_df['Perc Of Labeled Notes'] + ')'
perc_df.drop('Perc Of Labeled Notes', axis=1, inplace=True)
# RENAME COLUMN TO 'Labeled Notes (\%)'
perc_df.rename(columns={'Labeled Notes': 'Labeled Notes (\%)'}, inplace=True)

perc_df.reset_index(inplace=True, drop=True)
perc_df.index+=1
perc_df.index.name = 'Labeler'
# Put index and column in the same latex row / header
perc_df.reset_index(inplace=True)
# perc_df.index = [f"\\textit{{{idx}}}" for idx in perc_df.index]

perc_df.columns = [f"\\textbf{{{col}}}" for col in perc_df.columns]

perc_lat = perc_df.to_latex(index=False, float_format="%.1f")
print(perc_lat)

\begin{tabular}{rll}
\toprule
\textbf{Labeler} & \textbf{Labeled Notes (\%)} & \textbf{Tot Rec. Time} \\
\midrule
1 & 6,981 (21.4\%) &  2h 13' \\
2 & 11,041 (33.9\%) &  2h 56' \\
3 & 4,988 (15.3\%) &  1h 39' \\
4 & 4,534 (13.9\%) &  1h 36' \\
5 & 5,048 (15.5\%) &  1h 38' \\
\bottomrule
\end{tabular}

